1. Activate the virtual environment: 
source /Users/sofiamorenolasa/Desktop/TFG/.venv/bin/activate
2. Switch to the virtual environment

In [17]:
import pandas as pd

file_path = "../data/PARAMETROS_TJ2.csv"

# Use latin1 encoding
data = pd.read_csv(file_path, delimiter=";", encoding="latin1")

# Display the first few rows
print(data.head())

   N_DESCARGA       fecha   hora comentarioDesc comentarioExp configuracion  \
0         112  19971217.0  19:05            NaN           NaN           NaN   
1         113  19971217.0  19:09            NaN           NaN           NaN   
2         114  19971217.0  19:55            NaN           NaN           NaN   
3         115  19971218.0  11:08            NaN           NaN           NaN   
4         116  19971218.0  11:28            NaN           NaN           NaN   

   potencia_radiada  energia_diamagnetica  retraso_densidad_girotron  zeff  \
0               NaN                   NaN                        NaN   NaN   
1               NaN                   NaN                        NaN   NaN   
2               NaN                   NaN                        NaN   NaN   
3               NaN                   NaN                        NaN   NaN   
4               NaN                   NaN                        NaN   NaN   

   ...  IAccel_nominal_NBI2  tini_NBI2  longitud_pulso_n

/var/folders/b6/pb_stfjn0pb7pybgwlqjk5700000gn/T/ipykernel_87239/2676448733.py:6: DtypeWarning: Columns (52,55,96,132,133,144,145) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, delimiter=";", encoding="latin1")


Next, we sort the PARAMETROS_TJ2.csv file by the N_DESCARGA field and save the sorted data to a new file named PARAMETROS_TJ2_ORDENADOS.csv. This ensures the rows are ordered starting from the smallest N_DESCARGA value to the largest.

In [14]:
import pandas as pd

# Load the dataset
input_file_path = "../data/PARAMETROS_TJ2.csv"  # Replace with the correct path
output_file_path = "../data/PARAMETROS_TJ2_ORDENADOS.csv"

# Read the data
data = pd.read_csv(input_file_path, delimiter=";", encoding="latin1")

# Convert N_DESCARGA to numeric, coercing errors to NaN
data['N_DESCARGA'] = pd.to_numeric(data['N_DESCARGA'], errors='coerce')

# Drop rows where N_DESCARGA is NaN (invalid values)
data = data.dropna(subset=['N_DESCARGA'])

# Ensure N_DESCARGA is an integer
data['N_DESCARGA'] = data['N_DESCARGA'].astype(int)

# Sort by N_DESCARGA in ascending order
data_sorted = data.sort_values(by='N_DESCARGA')

# Save the sorted dataset to a new CSV file
data_sorted.to_csv(output_file_path, index=False, sep=";", encoding="latin1")

print(f"Sorted data saved to {output_file_path}")

/var/folders/b6/pb_stfjn0pb7pybgwlqjk5700000gn/T/ipykernel_87239/1307042932.py:8: DtypeWarning: Columns (52,55,96,132,133,144,145) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(input_file_path, delimiter=";", encoding="latin1")


Sorted data saved to ../data/PARAMETROS_TJ2_ORDENADOS.csv


In [ ]:
import pandas as pd
from transformers import pipeline

# Load the CSV file
file_path = "../data/PARAMETROS_TJ2_ORDENADOS.csv"  # Replace with your file path
data = pd.read_csv(file_path, delimiter=";", encoding="latin1", low_memory=False)

# Handle missing values and convert all columns to strings
data.fillna("N/A", inplace=True)
data["N_DESCARGA"] = data["N_DESCARGA"].astype(int)  # Ensure "N_DESCARGA" is treated as integers
data = data.astype(str)  # Convert the entire DataFrame to strings for TAPAS compatibility

# Reduce table size to relevant columns if needed
columns_to_keep = ["N_DESCARGA", "fecha", "hora", "comentarioDesc"]
data = data[columns_to_keep].sort_values(by="N_DESCARGA").reset_index(drop=True)

# Load the TAPAS pipeline with the base model
pipe = pipeline("table-question-answering", model="google/tapas-base-finetuned-wtq", device=-1)  # Force CPU usage


# Binary search algorithm to find the row with the target N_DESCARGA
def binary_search_discharge(data, target):
    low, high = 0, len(data) - 1
    
    while low <= high:
        mid = (low + high) // 2
        mid_value = int(data.iloc[mid]["N_DESCARGA"])  # Ensure mid_value is treated as an integer
        
        if mid_value == target:
            return data.iloc[[mid]]  # Return the row as a DataFrame
        elif mid_value < target:
            low = mid + 1
        else:
            high = mid - 1
    
    return pd.DataFrame()  # Return empty DataFrame if not found


# Define questions
questions = [
    "What is the fecha for N_DESCARGA 4?",
    "What is the hora for N_DESCARGA 6458?",
    "What is the comentarioDesc for N_DESCARGA 12?"
]

# Process each question
print("\nResults:")
for question in questions:
    try:
        # Extract the N_DESCARGA value from the question
        query_value = int(question.split()[-1].strip("?"))

        # Use binary search to find the relevant row
        filtered_table = binary_search_discharge(data, query_value)

        # Check if the filtered table is empty
        if filtered_table.empty:
            print(f"Question: {question}")
            print("Answer: No matching data found in the table.\n")
            continue

        # Run the pipeline with the filtered table
        answer = pipe(table=filtered_table, query=question)
        print(f"Question: {question}")
        print(f"Answer: {answer['answer']}\n")

    except ValueError as e:
        print(f"Error parsing question: {e}")
    except Exception as e:
        print(f"Error during TAPAS processing for question '{question}': {e}\n")

/var/folders/b6/pb_stfjn0pb7pybgwlqjk5700000gn/T/ipykernel_47698/3740190966.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'N/A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.fillna("N/A", inplace=True)
Device set to use cpu



Results:
Question: What is the fecha for N_DESCARGA 4?
Answer: No matching data found in the table.

Question: What is the hora for N_DESCARGA 6458?
Answer: No matching data found in the table.

Question: What is the comentarioDesc for N_DESCARGA 12?
Answer: No matching data found in the table.



In [ ]:
import pandas as pd
from transformers import pipeline

# Load the CSV file
file_path = "../data/PARAMETROS_TJ2_ORDENADOS.csv"  # Replace with your file path
data = pd.read_csv(file_path, delimiter=";", encoding="latin1", low_memory=False)

# Handle missing values and convert all columns to strings
data.fillna("N/A", inplace=True)
data["N_DESCARGA"] = data["N_DESCARGA"].astype(int)  # Ensure "N_DESCARGA" is treated as integers
data = data.astype(str)  # Convert the entire DataFrame to strings for TAPAS compatibility

# Reduce table size to relevant columns if needed
columns_to_keep = ["N_DESCARGA", "fecha", "hora", "comentarioDesc"]
data = data[columns_to_keep].sort_values(by="N_DESCARGA").reset_index(drop=True)

# Load the TAPAS pipeline with the base model
pipe = pipeline("table-question-answering", model="google/tapas-base-finetuned-wtq", device=-1)  # Force CPU usage


# Binary search algorithm to find the row with the target N_DESCARGA
def binary_search_discharge(data, target):
    low, high = 0, len(data) - 1
    
    while low <= high:
        mid = (low + high) // 2
        mid_value = int(data.iloc[mid]["N_DESCARGA"])  # Ensure mid_value is treated as an integer
        
        if mid_value == target:
            return data.iloc[[mid]]  # Return the row as a DataFrame
        elif mid_value < target:
            low = mid + 1
        else:
            high = mid - 1
    
    return pd.DataFrame()  # Return empty DataFrame if not found


# Define questions
questions = [
    "What is the fecha for N_DESCARGA 4?",
    "What is the hora for N_DESCARGA 6458?",
    "What is the comentarioDesc for N_DESCARGA 12?"
]

# Process each question
print("\nResults:")
for question in questions:
    try:
        # Extract the N_DESCARGA value from the question
        query_value = int(question.split()[-1].strip("?"))

        # Use binary search to find the relevant row
        filtered_table = binary_search_discharge(data, query_value)

        # Check if the filtered table is empty
        if filtered_table.empty:
            print(f"Question: {question}")
            print("Answer: No matching data found in the table.\n")
            continue

        # Run the pipeline with the filtered table
        answer = pipe(table=filtered_table, query=question)
        print(f"Question: {question}")
        print(f"Answer: {answer['answer']}\n")

    except ValueError as e:
        print(f"Error parsing question: {e}")
    except Exception as e:
        print(f"Error during TAPAS processing for question '{question}': {e}\n")

/var/folders/b6/pb_stfjn0pb7pybgwlqjk5700000gn/T/ipykernel_47698/3740190966.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'N/A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.fillna("N/A", inplace=True)
Device set to use cpu



Results:
Question: What is the fecha for N_DESCARGA 4?
Answer: No matching data found in the table.

Question: What is the hora for N_DESCARGA 6458?
Answer: No matching data found in the table.

Question: What is the comentarioDesc for N_DESCARGA 12?
Answer: No matching data found in the table.



Hay que borrar los datos repetidos, hay muchos donde la hora y la fecha son iguales, aunque el numero de descarga sea diferente, así que habría que dejar solo uno. 